# Проект по SQL 

**Описание:** компания решила купить крупный сервис для чтения книг по подписке. 

**Задача:** проанализировать базу данных.


В нашем распоряжении информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Используя SQL запросы ответить на вопросы:

- 1. Сколько книг вышло после 1 января 2000 года;
- 2. Какое количество обзоров и средняя оценка для каждой книги посчитайте;
- 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- 4. Определить автора с самой высокой средней оценкой книг(только книги с 50 и более оценками);
- 5. Какое среднее количество обзоров от пользователей(которые поставили больше 50 оценок).

**Ход действий**
 1. Импорт необходимых библиотек
 2. Подключение к базе данных
     - подключение и проверка данных
 3. Анализ базы данных
     - ответы на вопросы
 4. Выводы

## Импорт библиотек и загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

## Подключаемся к БД и делаем SQL запрос

In [2]:
#словарь для подключения к БД
db_config = {'user': '*****', # имя пользователя
            'pwd': '*****', # пароль
            'host': '*****',
            'port': '*****', # порт подключения
            'db': '*****'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
#Собориаем подключение в строку:
engine = create_engine(connection_string)

### Проверка подключения и выполнения SQL запросов:

In [3]:
#Пишем SQL запрос
query = ''' SELECT *
            FROM books
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Подключились посмотрим количество данных всего у нас в базе:

Сколько всего книг:

In [4]:
#Пишем SQL запрос
query = ''' SELECT COUNT(book_id)
            FROM books
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,count
0,1000


Сколько всего авторов:

In [5]:
#Пишем SQL запрос
query = ''' SELECT COUNT(DISTINCT author_id)
            FROM books
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,count
0,636


Сколько всего издательств:

In [6]:
#Пишем SQL запрос
query = ''' SELECT COUNT(publisher)
            FROM publishers
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,count
0,340


Сколько всего пользователей:

In [7]:
#Пишем SQL запрос
query = ''' SELECT COUNT(DISTINCT username)
            FROM ratings
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,count
0,160


Сколько всего оценок книг:

In [8]:
#Пишем SQL запрос
query = ''' SELECT COUNT(*)
            FROM ratings
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,count
0,6456


Сколько всего обзоров книг:

In [9]:
#Пишем SQL запрос
query = ''' SELECT COUNT(*)
            FROM reviews
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,count
0,2793


# Анализ базы данных

Ответы на наши вопросы:

### Сколько книг вышло после 1 января 2000 года;

In [10]:
#Пишем SQL запрос
query = ''' SELECT COUNT(*)
            FROM books
            WHERE publication_date > '2000-01-01'
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books

,count
0,819


### Какое количество обзоров и средняя оценка для каждой книги посчитайте (войдут только имеющие оценки);

In [11]:
#Пишем SQL запрос
query = '''SELECT bo.book_id AS book_id_main,
                  bo.title,
                  COUNT (DISTINCT re.review_id) AS count_review,
                  ROUND (AVG(ra.rating),2)
           FROM books as bo
           LEFT JOIN ratings AS ra ON bo.book_id=ra.book_id
           LEFT JOIN reviews AS re ON bo.book_id=re.book_id
           GROUP BY book_id_main
           ORDER BY count_review DESC
           '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)
display(df_books.shape[0])
df_books.head(20)

1000

,book_id_main,title,count_review,round
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
5,696,The Da Vinci Code (Robert Langdon #2),6,3.83
6,627,The Alchemist,6,3.79
7,750,The Hobbit or There and Back Again,6,4.13
8,656,The Book Thief,6,4.26
9,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.08


Посмотрим на лидеров по рейтингу и кол-во оценок (кол-во оценок более 150)

### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;

In [12]:
#Пишем SQL запрос
query = ''' SELECT p.publisher,
                   COUNT(b.book_id) as num_books
            FROM books AS b
            JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY num_books DESC
            LIMIT 1
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,publisher,num_books
0,Penguin Books,42


### Определить автора с самой высокой средней оценкой книг(только книги с 50 и более оценками);

In [13]:
#Пишем SQL запрос
query = ''' WITH best_books as (SELECT r.book_id,
                                       COUNT(rating) as num_rating
                                FROM ratings AS r
                                GROUP BY r.book_id
                                HAVING COUNT(rating) > 50
                                ORDER BY num_rating DESC)
            
            SELECT a.author,
                   AVG(r.rating) AS avg_rating
            FROM best_books as bb
            JOIN books as b on bb.book_id = b.book_id
            JOIN authors as a on b.author_id = a.author_id
            JOIN ratings as r on bb.book_id = r.book_id
            GROUP BY a.author
            ORDER BY avg_rating DESC
            LIMIT 1
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books.head()

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


### Какое среднее количество обзоров от пользователей(которые поставили больше 50 оценок).

In [14]:
#Пишем SQL запрос
query = ''' WITH best_users as (SELECT r.username,
                                       COUNT(rating) as num_rating
                                FROM ratings AS r
                                GROUP BY r.username
                                HAVING COUNT(rating) > 50
                                ORDER BY num_rating DESC),
                                
               counts_review as (SELECT re.username,
                                       COUNT(re.text) as count_review
                                FROM best_users as bu
                                JOIN reviews AS re ON bu.username = re.username
                                GROUP BY re.username)
           SELECT AVG(count_review)
           FROM counts_review
        '''
#Сохраняем в переменную загруженную БД
df_books = pd.io.sql.read_sql(query, con = engine)

df_books

,avg
0,24.333333


# Выводы

ОБЩЕЕ:
 - Всего книг: **1000**
 - Всего авторов: **636**
 - Всего издательств: **340**
 - Всего пользователей: **160**
 - Всего оценок книг: **6456**
 - Всего обзоров книг: **2793**
    
Ответы на вопросы:
 1. 819 книг вышло после 1 января 2000 года
 2. Количество обзоров и средняя оценка для каждой книги:
     - Лидер по оценкам `Twilight` (Twilight №1) кол-во обзоров: **7**, средняя оценка: **3.66** 
     - Лидер по оценкам и рейтингу `Harry Potter and the Prisoner of Azkaban` кол-во обзоров: **6**, средняя оценка: **4.41**
 3. Издательство лидер по кол-ву изданых книг(не менее 50 страниц в книге) это `Penguin Books` (**42** книги)
 4. Автор с самой высокой средней оценкой книг `J.K. Rowling / Mary GrandPré` (**4.287**)
 5. **24,33** - среднее количество обзоров от пользователей (которые поставили больше 50 оценок)